In [1]:
# Dependencies
import requests
from pprint import pprint
import json
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
from flask_cors import CORS

# base URL
base_url = 'https://api.data.abs.gov.au/data/'

# GetData Required Parameters, explore.data.abs.gov.au for finding parameter values for a sepcific dataset
dataflowIdentifier = "ABS,RES_DWELL_ST,1.0.0"
dataKey = ""

# GetData Query Parameters, the default format is XML, uses SDMX
format = "format=jsondata"
detail = "detail=dataonly"

# Request URL
url = base_url + f"{dataflowIdentifier}/{dataKey}?{detail}&{format}"
url

'https://api.data.abs.gov.au/data/ABS,RES_DWELL_ST,1.0.0/?detail=dataonly&format=jsondata'

In [2]:
response = requests.get(url)

if response.status_code == 200:
    # Successful API call
    json_data = response.json()
    # Do something with the data
    pprint(json_data)
else:
    # Handle unsuccessful API call
    print("Error: Failed to fetch data from ABS website.")

{'data': {'dataSets': [{'action': 'Information',
                        'annotations': [0, 1, 2, 3, 4, 5],
                        'links': [{'rel': 'DataStructure',
                                   'urn': 'urn:sdmx:org.sdmx.infomodel.datastructure.DataStructure=ABS:RES_DWELL_ST(1.0.0)'}],
                        'series': {'0:0:0': {'annotations': [],
                                             'attributes': [None, None],
                                             'observations': {'0': [253939.4],
                                                              '1': [257468.3],
                                                              '10': [277542.8],
                                                              '11': [278665.4],
                                                              '12': [279057.2],
                                                              '13': [287533],
                                                              '14': [289528],
               

                                                              '14': [237.1],
                                                              '15': [237.6],
                                                              '16': [238.1],
                                                              '17': [238.9],
                                                              '18': [239.3],
                                                              '19': [239.8],
                                                              '2': [231.6],
                                                              '20': [240.2],
                                                              '21': [240.7],
                                                              '22': [241.1],
                                                              '23': [241.5],
                                                              '24': [242],
                                                              '25': [242.5],
  

In [3]:
# find the data using the path
series = json_data['data']['dataSets'][0]['series']

In [4]:
# locate series of interest
mean_aus_dwelling = series['4:3:0']['observations']

In [5]:
# Convert JSON data into a DataFrame
aus_df = pd.DataFrame.from_dict(mean_aus_dwelling, orient='index', columns=['mean price of residential dwelling'])
aus_df.head()

,mean price of residential dwelling
0,490.8
1,486.9
2,488.6
3,489.9
4,486.3


In [6]:
# Create a PeriodIndex for the time period with a frequency of 'Q' for quarters
time_period = pd.date_range(start='2011-07-01', periods=45, freq='Q')

# Convert the date to YYYY-MM-DD format
time_period = time_period.strftime('%Y-%m-%d')

# Add the date column to the DataFrame
aus_df['date'] = time_period

# Rename the columns
aus_df = aus_df.rename(columns={'mean price of residential dwelling': 'mean_price'})
aus_df.head()

,mean_price,date
0,490.8,2011-09-30
1,486.9,2011-12-31
2,488.6,2012-03-31
3,489.9,2012-06-30
4,486.3,2012-09-30


In [10]:
# connect to cloud database using SQLAlchemy
protocol = 'postgresql'
username = 'anshumanp'
password = 'Anshuman123'
host = 'mydbinstance-anshu.c3y42tkkiinb.us-east-2.rds.amazonaws.com'
port = 5432
database_name = 'rates_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

# Confirm tables
insp.get_table_names()

['interest_rates', 'aus_dwelling_mean_ans', 'aus_dwelling_mean']

In [11]:
# Load DataFrame into Postgres Database
aus_df.to_sql(name='aus_dwelling_mean', con=engine, if_exists='append', index=False)

# Test Database with SQL Query
pd.read_sql_query('SELECT * from aus_dwelling_mean LIMIT 10', con=engine).head()

,id,date,mean_price
0,1,2011-09-30,490.8
1,2,2011-12-31,486.9
2,3,2012-03-31,488.6
3,4,2012-06-30,489.9
4,5,2012-09-30,486.3
